In [1]:
import os
import duckdb
import pandas as pd
import yaml
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import TargetEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

import pandas as pd
import numpy as np

from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import classification_report, accuracy_score, mean_absolute_error, mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import RandomizedSearchCV

### ER - BEGIN ###
import pickle
### ER - END ###

In [2]:
# file_path1 = r"/home/kmohseni/private/DSC288R_Capstone/data/final/X_test_enc.parquet"
# file_path2 = r"/home/kmohseni/private/DSC288R_Capstone/data/final/X_train_enc.parquet"
# file_path3 = r"/home/kmohseni/private/DSC288R_Capstone/data/final/y_test.parquet"
# file_path4 = r"/home/kmohseni/private/DSC288R_Capstone/data/final/y_train.parquet"

### ER - BEGIN ###
file_path1 = "../data/final/X_test_enc.parquet"
file_path2 = "../data/final/X_train_enc.parquet"
file_path3 = "../data/final/y_test.parquet"
file_path4 = "../data/final/y_train.parquet"
### ER - END ###

X_test_enc = pd.read_parquet(file_path1)
X_train_enc = pd.read_parquet(file_path2)
y_test = pd.read_parquet(file_path3)
y_train = pd.read_parquet(file_path4)

In [3]:
print(X_train_enc.columns)

Index(['FlightDate', 'DayOfWeek', 'Month', 'Airline', 'Origin', 'Dest',
       'Cancelled', 'Diverted', 'DivAirportLandings', 'CRSDepTime', 'DepTime',
       'DepDelay', 'CRSArrTime', 'ArrTime', 'ArrDelay', 'ArrDel15', 'AirTime',
       'Distance', 'TaxiOut', 'WheelsOff', 'WheelsOn', 'TaxiIn',
       'DelayCategory', 'AirTimeCategory', 'TimeofDay', 'Month_sin',
       'Month_cos', 'DayOfWeek_sin', 'DayOfWeek_cos', 'DayOfYear',
       'DayOfYear_sin', 'DayOfYear_cos', 'Holiday_Indicator', 'Near_Holiday',
       'Origin_PRCP', 'Origin_SNOW', 'Origin_SNWD', 'Origin_TMAX',
       'Origin_TMIN', 'Dest_PRCP', 'Dest_SNOW', 'Dest_SNWD', 'Dest_TMAX',
       'Dest_TMIN', 'weekly_avg_origin_tmin', 'weekly_avg_dest_tmin',
       'monthly_avg_origin_tmin', 'monthly_avg_dest_tmin',
       'weekly_avg_origin_tmax', 'weekly_avg_dest_tmax',
       'monthly_avg_origin_tmax', 'monthly_avg_dest_tmax',
       'weekly_avg_origin_prcp', 'weekly_avg_dest_prcp',
       'monthly_avg_origin_prcp', 'monthly_avg_d

In [4]:
print(X_test_enc.columns)

Index(['FlightDate', 'DayOfWeek', 'Month', 'Airline', 'Origin', 'Dest',
       'Cancelled', 'Diverted', 'DivAirportLandings', 'CRSDepTime', 'DepTime',
       'DepDelay', 'CRSArrTime', 'ArrTime', 'ArrDelay', 'ArrDel15', 'AirTime',
       'Distance', 'TaxiOut', 'WheelsOff', 'WheelsOn', 'TaxiIn',
       'DelayCategory', 'AirTimeCategory', 'TimeofDay', 'Month_sin',
       'Month_cos', 'DayOfWeek_sin', 'DayOfWeek_cos', 'DayOfYear',
       'DayOfYear_sin', 'DayOfYear_cos', 'Holiday_Indicator', 'Near_Holiday',
       'Origin_PRCP', 'Origin_SNOW', 'Origin_SNWD', 'Origin_TMAX',
       'Origin_TMIN', 'Dest_PRCP', 'Dest_SNOW', 'Dest_SNWD', 'Dest_TMAX',
       'Dest_TMIN', 'weekly_avg_origin_tmin', 'weekly_avg_dest_tmin',
       'monthly_avg_origin_tmin', 'monthly_avg_dest_tmin',
       'weekly_avg_origin_tmax', 'weekly_avg_dest_tmax',
       'monthly_avg_origin_tmax', 'monthly_avg_dest_tmax',
       'weekly_avg_origin_prcp', 'weekly_avg_dest_prcp',
       'monthly_avg_origin_prcp', 'monthly_avg_d

In [5]:
# Drop unnecessary or repetitive columns
X_train_class = X_train_enc.drop(columns=['TimeofDay', 'AirTimeCategory', 'FlightDate', 'Cancelled', 'Diverted', 'DepDelay', 'ArrDelay', 'ArrDel15', 'DelayCategory'])
X_train_reg = X_train_enc.drop(columns=['TimeofDay', 'AirTimeCategory', 'FlightDate', 'Cancelled', 'Diverted', 'DepDelay', 'ArrDelay', 'ArrDel15', 'DelayCategory'])

X_test_class = X_test_enc.drop(columns=['TimeofDay', 'AirTimeCategory', 'FlightDate', 'Cancelled', 'Diverted', 'DepDelay', 'ArrDelay', 'ArrDel15', 'DelayCategory'])
X_test_reg = X_test_enc.drop(columns=['TimeofDay', 'AirTimeCategory', 'FlightDate', 'Cancelled', 'Diverted', 'DepDelay', 'ArrDelay', 'ArrDel15', 'DelayCategory'])

y_train_class = y_train
y_train_reg = X_train_enc['DepDelay']

y_test_class = y_test
y_test_reg = X_test_enc['DepDelay']

In [6]:
print(X_train_class.columns)
print(X_train_reg.columns)
print(X_test_class.columns)
print(X_test_reg.columns)

Index(['DayOfWeek', 'Month', 'Airline', 'Origin', 'Dest', 'DivAirportLandings',
       'CRSDepTime', 'DepTime', 'CRSArrTime', 'ArrTime', 'AirTime', 'Distance',
       'TaxiOut', 'WheelsOff', 'WheelsOn', 'TaxiIn', 'Month_sin', 'Month_cos',
       'DayOfWeek_sin', 'DayOfWeek_cos', 'DayOfYear', 'DayOfYear_sin',
       'DayOfYear_cos', 'Holiday_Indicator', 'Near_Holiday', 'Origin_PRCP',
       'Origin_SNOW', 'Origin_SNWD', 'Origin_TMAX', 'Origin_TMIN', 'Dest_PRCP',
       'Dest_SNOW', 'Dest_SNWD', 'Dest_TMAX', 'Dest_TMIN',
       'weekly_avg_origin_tmin', 'weekly_avg_dest_tmin',
       'monthly_avg_origin_tmin', 'monthly_avg_dest_tmin',
       'weekly_avg_origin_tmax', 'weekly_avg_dest_tmax',
       'monthly_avg_origin_tmax', 'monthly_avg_dest_tmax',
       'weekly_avg_origin_prcp', 'weekly_avg_dest_prcp',
       'monthly_avg_origin_prcp', 'monthly_avg_dest_prcp',
       'weekly_avg_origin_snow', 'weekly_avg_dest_snow',
       'monthly_avg_origin_snow', 'monthly_avg_dest_snow',
       'wee

# Classification

### Random Forest

In [ ]:
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train_class, y_train_class)

/opt/conda/lib/python3.11/site-packages/sklearn/base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


In [ ]:
### ER - BEGIN ###
with open("../models/rf_class.pkl", "wb") as file:
    pickle.dump(rf_classifier, file)

In [7]:
with open("../models/rf_class.pkl", "rb") as file:
    rf_classifier = pickle.load(file)
### ER - END ###

In [8]:
y_pred_rf = rf_classifier.predict(X_test_class)

print("Random Forest Classification Report:")
print(classification_report(y_test_class, y_pred_rf))

Random Forest Classification Report:
              precision    recall  f1-score   support

         0.0       0.97      0.97      0.97    763012
         1.0       0.97      0.97      0.97    763284

    accuracy                           0.97   1526296
   macro avg       0.97      0.97      0.97   1526296
weighted avg       0.97      0.97      0.97   1526296



### Histogram-Based Gradient Boosting Classification Tree

In [9]:
# Initialize the HistGradientBoosting classifier
hgb_classifier = HistGradientBoostingClassifier(random_state=42)

# Train the model
hgb_classifier.fit(X_train_class, y_train_class)

/opt/conda/lib/python3.11/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


HistGradientBoostingClassifier(random_state=42)

In [10]:
### ER - BEGIN ###
with open("../models/hgb_class.pkl", "wb") as file:
    pickle.dump(hgb_classifier, file)

In [9]:
with open("../models/hgb_class.pkl", "rb") as file:
    hgb_classifier = pickle.load(file)
### ER - END ###

In [10]:
# Make predictions
y_pred_hgb = hgb_classifier.predict(X_test_class)

In [11]:
# Print classification report
print("HistGradientBoosting Classification Report:")
print(classification_report(y_test_class, y_pred_hgb))

HistGradientBoosting Classification Report:
              precision    recall  f1-score   support

         0.0       0.98      0.98      0.98    763012
         1.0       0.98      0.98      0.98    763284

    accuracy                           0.98   1526296
   macro avg       0.98      0.98      0.98   1526296
weighted avg       0.98      0.98      0.98   1526296



# Regression

### Linear Regression

In [7]:
# Impute missing values with the median (or use 'mean' or 'most_frequent')
imputer = SimpleImputer(strategy="median")

# Fit and transform the training set
X_train_reg = imputer.fit_transform(X_train_reg)
X_test_reg = imputer.transform(X_test_reg)  # Use same imputer for test data

In [8]:
X_train_reg = X_train_reg.astype(np.float32)
X_test_reg = X_test_reg.astype(np.float32)
y_train_reg = y_train_reg.astype(np.float32)
y_test_reg = y_test_reg.astype(np.float32)

In [9]:
lin_reg = LinearRegression()
lin_reg.fit(X_train_reg, y_train_reg)

LinearRegression()

In [10]:
### ER - BEGIN ###
with open("../models/lin_reg.pkl", "wb") as file:
    pickle.dump(lin_reg, file)

In [9]:
with open("../models/lin_reg.pkl", "rb") as file:
    lin_reg = pickle.load(file)
### ER - END ###

In [10]:
y_pred_linreg = lin_reg.predict(X_test_reg)

print("Linear Regression MAE:", mean_absolute_error(y_test_reg, y_pred_linreg))
print("Linear Regression RMSE:", np.sqrt(mean_squared_error(y_test_reg, y_pred_linreg)))
print("Linear Regression r2-score:", r2_score(y_test_reg, y_pred_linreg))

Linear Regression MAE: 37.858036
Linear Regression RMSE: 74.57018
Linear Regression r2-score: 0.04447780530426271


### Hyperparameter Tuning

In [13]:
# Define hyperparameter space for Linear Regression
param_dist_lr = {
    "fit_intercept": [True, False]
}

In [14]:
# Initialize model
lr = LinearRegression()

# Use RandomizedSearchCV instead of GridSearchCV for efficiency
random_search_lr = RandomizedSearchCV(
    lr, param_distributions=param_dist_lr, cv=3, scoring="r2",
    n_iter=5, n_jobs=1, random_state=42
)

In [15]:
# Train model
random_search_lr.fit(X_train_reg, y_train_reg)

/opt/conda/lib/python3.11/site-packages/sklearn/model_selection/_search.py:318: UserWarning: The total space of parameters 2 is smaller than n_iter=5. Running 2 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


RandomizedSearchCV(cv=3, estimator=LinearRegression(), n_iter=5, n_jobs=1,
                   param_distributions={'fit_intercept': [True, False]},
                   random_state=42, scoring='r2')

In [17]:
### ER - BEGIN ###
with open("../models/lin_reg_random_search.pkl", "wb") as file:
    pickle.dump(random_search_lr, file)

In [11]:
with open("../models/lin_reg_random_search.pkl", "rb") as file:
    random_search_lr = pickle.load(file)
### ER - END ###

In [12]:
# Best parameters and score
print("Best Linear Regression Parameters:", random_search_lr.best_params_)
print("Best Linear Regression Score:", random_search_lr.best_score_)

Best Linear Regression Parameters: {'fit_intercept': True}
Best Linear Regression Score: 0.045121278048631606


In [13]:
# Evaluate on test data
y_pred_lr = random_search_lr.best_estimator_.predict(X_test_reg)
print("Linear Regression R2 Score on Test Set:", r2_score(y_test_reg, y_pred_lr))

Linear Regression R2 Score on Test Set: 0.04447780530426271
